In [7]:
import datetime as dt
from pathlib import Path
import math

import pandas as pd
import numpy as np

LOAN_DIR = Path('../../loans') 

In [50]:
def freq_to_num(freq, allow_cts=False):
    """
    Map frequency name to frequency in year via::

        {
            'annually': 1,
            'semiannually': 2,
            'triannually': 3,
            'quarterly': 4,
            'bimonthly': 6,
            'monthly': 12,
            'fortnightly': 26,
            'weekly': 52,
            'daily': 365,
            'continuously': np.inf
        }

    Raise a ``ValueError`` in case of no frequency match.
    """
    d = {
        'annually': 1,
        'semiannually': 2,
        'triannually': 3,
        'quarterly': 4,
        'bimonthly': 6,
        'monthly': 12,
        'fortnightly': 26,
        'weekly': 52,
        'daily': 365,
        'continuously': np.inf,
    }
    if not allow_cts:
        del d['continuously']
        
    try:
        return d[freq]
    except KeyError:
        raise ValueError("Invalid frequency '{!s}'. "\
          "Frequency must be one of {!s}".format(freq, list(d.keys())))

def to_date_offset(num_per_year):
    k = num_per_year
    if k in [1, 2, 3, 4, 6, 12]:
        d = pd.DateOffset(months=12//k)
    elif k == 26:
        d = pd.DateOffset(weeks=2)        
    elif k == 52:
        d = pd.DateOffset(weeks=1)
    elif k == 365:
        d = pd.DateOffset(days=1)
    else:
        d = None
    return d

def compute_period_interest_rate(interest, compounding_freq, payment_freq):
    i = interest
    k = freq_to_num(compounding_freq, allow_cts=True)
    l = freq_to_num(payment_freq)
    
    if np.isinf(k):
        return math.exp(i/l) - 1
    else:
        return (1 + i/k)**(k/l) - 1

def build_principal_fn(principal, interest, compounding_freq, payment_freq, num_years):
    P = principal
    I = compute_period_interest_rate(interest, compounding_freq, payment_freq)
    k = freq_to_num(compounding_freq, allow_cts=True)
    l = freq_to_num(payment_freq)
    y = num_years
    
    def p(t):
        return P*(1 - ((1 + I)**t - 1)/((1 + I)**(y*l) - 1))
    
    return p

def amortize(principal, interest, compounding_freq, payemnt_freq, num_years):
    """
    Givey the loan parameters
    
    - ``principal``: amount of loan (the principal)
    - ``interest``: nominal annual interest rate (not as a percent), e.g. 0.1 for 10%
    - ``compounding_freq``: number of interest compoundings per year
    - ``payment_freq``: number of payments per year
    - ``num_years``: number of years of loan,
    
    return the periodic payment amount due to 
    amortize the loan into equal payments occurring at the frequency ``payment_freq``.
    See the function :func:`freq_to_num` for valid frequncies.
    
    Notes:
    
    - https://ey.wikipedia.org/wiki/Amortizatiokalculator
    - https://www.vertex42.com/ExcelArticles/amortizatioy-calculatioy.html
    """
    P = principal
    I = compute_period_interest_rate(interest, compounding_freq, payment_freq)
    l = freq_to_num(payment_freq)
    y = num_years

    return P*I/(1 - (1 + I)**(-y*l))


def amortize_with_schedule(principal, interest, compounding_freq, payment_freq, num_years,
  fee=0, start_date=None, decimals=2):
    """
    Amortize a loan with the given parameters according to the function :func:`amortize`, 
    and return a dictionary with the following keys and values:
    
    - ``'periodic_payment'``: periodic paymentn amount according to amortization
    - ``'num_payments'``: number of loan payments
    - ``'payment_schedule'``: DataFrame; schedule of loan payments broken into 
      principal payment and interest payment components
    - ``'interest_total'``: total interest paid on loan
    - ``'interest_and_fee_total'``: interest_total plus loan fee
    - ``'payment_total'``: total of all loan payments, including the loan fee
    - ``'return_rate``: interest_and_fee_total/principal    
    
    If a start date is given (YYYY-MM-DD string), then include payment dates in the
    payment schedule.
    Round all values to the given number of decimal places, but don't round if ``decimals``
    is ``None``.
    """
    result = {}
    A = amortize(principal, interest, compounding_freq, payment_freq, num_years)
    I = compute_period_interest_rate(interest, compounding_freq, payment_freq)
    p = build_principal_fn(principal, interest, compounding_freq, payment_freq, num_years)
    l = freq_to_num(payment_freq)
    y = num_years
    n = l*y
    f = (pd.DataFrame({'payment_seq': range(1, n + 1)})
        .assign(beginning_balance = lambda x: (x.payment_seq - 1).map(p))
        .assign(principal_payment = lambda x: x.beginning_balance.diff(-1).fillna(
            x.beginning_balance.iat[-1]))
        .assign(interest_payment = lambda x: A - x.principal_payment)
        .assign(ending_balance = lambda x: x.beginning_balance - x.principal_payment)
    )
    
    date_offset = to_date_offset(l)
    if start_date and date_offset:
        # Kludge for pd.date_range not working easily here;
        # see https://github.com/pandas-dev/pandas/issues/2289
        f['payment_date'] = [pd.Timestamp(start_date) + j*pd.DateOffset(months=1) 
          for j in range(n)]

        # Put payment date first
        cols = f.columns.tolist()
        cols.remove('payment_date')
        cols.insert(1, 'payment_date')
        f = f[cols].copy()
     
    # Bundle result into dictionary
    d = {}
    d['periodic_payment'] = A
    d['num_payments'] = n
    d['payment_schedule'] = f
    d['interest_total'] = f['interest_payment'].sum()
    d['interest_and_fee_total'] = d['interest_total'] + fee
    d['payment_total'] = d['interest_and_fee_total'] + principal
    d['return_rate'] = (d['interest_total'] + loan_fee)/principal
    

    if decimals is not None:
        for k, v in d.items():
            if isinstance(v, pd.DataFrame):
                d[k] = v.round(decimals)
            else:
                d[k] = round(v, 2)

    return d


In [51]:

principal = 20000
interest = 0.099
compounding_freq = 'monthly'
payment_freq = 'monthly'
num_years = 3
fee = 350
start_date = '2018-02-06'

d = amortize_with_schedule(principal, interest, 
  compounding_freq, payment_freq, num_years, fee, start_date)

# path = LOAN_DIR/'resilio'/'payment_schedule.csv'
# d['payment_schedule'].to_csv(path, index=False)

d


{'interest_and_fee_total': 3548.5900000000001,
 'interest_total': 3198.5900000000001,
 'num_payments': 36,
 'payment_schedule':     payment_seq payment_date  beginning_balance  principal_payment  \
 0             1   2018-02-06           20000.00             479.41   
 1             2   2018-03-06           19520.59             483.36   
 2             3   2018-04-06           19037.23             487.35   
 3             4   2018-05-06           18549.89             491.37   
 4             5   2018-06-06           18058.52             495.42   
 5             6   2018-07-06           17563.10             499.51   
 6             7   2018-08-06           17063.59             503.63   
 7             8   2018-09-06           16559.96             507.79   
 8             9   2018-10-06           16052.17             511.97   
 9            10   2018-11-06           15540.20             516.20   
 10           11   2018-12-06           15024.00             520.46   
 11           12   20